# Export YOLOv10n to ONNX — Colab
This notebook walks through exporting a YOLO `.pt` model to ONNX using a Colab environment (GPU recommended). Follow the cells in order: install dependencies, upload model, export (via helper script or direct API), validate ONNX, and download the result.

**Select Runtime → Change runtime type → Hardware accelerator: GPU**

## Section 1 — Setup & Imports
Install required packages and import standard libraries. Run the cell below to set up the Colab environment.

In [ ]:
# Install dependencies (run this cell)
!pip install --upgrade pip
!pip install ultralytics onnx onnxruntime onnxsim --quiet

## Section 2 — Environment & dependency checks
Verify Python version and that key packages are available.

In [ ]:
# Quick environment checks
import sys
print('Python', sys.version)

# show installed ultralytics if present
!python -c "import ultralytics; print('ultralytics', ultralytics.__version__)" || echo 'ultralytics not installed'

# List packages of interest
!python -m pip show onnx onnxruntime onnxsim || true

## Section 3 — Upload/YAML & inspect model
Upload a `.pt` model from your machine or download it via URL. The cell below uses `files.upload()` to let you select a file interactively.

In [ ]:
# Upload .pt model interactively
from google.colab import files
uploaded = files.upload()
for fname in uploaded:
    print('Uploaded:', fname)

# Optionally check filesize and list
import os
for fname in uploaded:
    print(fname, os.path.getsize(fname))

## Sections 4-6 — Preprocessing, Features, EDA (Placeholders)
These sections are included as a template for broader data workflows. For model export they are typically not required, but we keep short examples and placeholders to show how you'd document and test these steps.

## Section 7 — Export model to ONNX (helper script)
Use the repo's helper export script to convert your uploaded `.pt` to `.onnx`.

Run the cell below to run: `python scripts/export_yolo_to_onnx.py --model yolov10n.pt --output yolov10n.onnx --opset 12 --simplify` (replace filenames as needed).

In [ ]:
# Run the helper script (adjust arguments if your model has another name)
!python scripts/export_yolo_to_onnx.py --model yolov10n.pt --output yolov10n.onnx --opset 12 --simplify

In [ ]:
# Direct export via ultralytics API (alternative)
try:
    from ultralytics import YOLO
    print('ultralytics available:', YOLO)
    model = YOLO('yolov10n.pt')
    print('Exporting using ultralytics API...')
    model.export(format='onnx', opset=12, imgsz=640, simplify=True)
    print('API export finished.')
except Exception as e:
    print('Ultralytics API export failed:', e)
    print('You can still try the helper script or adjust opset/simplify.')

## Section 8 — Validate ONNX
Check the exported ONNX file with `onnx.checker` and ensure `onnxruntime` can load it.

In [ ]:
# ONNX validation
import onnx
import onnxruntime as ort

onnx_path = 'yolov10n.onnx'
try:
    model = onnx.load(onnx_path)
    onnx.checker.check_model(model)
    print('ONNX check: OK')
    sess = ort.InferenceSession(onnx_path)
    print('onnxruntime providers:', sess.get_providers())
except Exception as e:
    print('ONNX validation failed:', e)
    print('Check the export output and try different opset/simplify settings')

## Section 9 — Save & export artifacts
Download the ONNX or copy it to your Google Drive for later use.

In [ ]:
# Download the ONNX file to your local machine
from google.colab import files
files.download('yolov10n.onnx')

# OR: copy to Google Drive (uncomment to use)
# from google.colab import drive
# drive.mount('/content/drive')
# !cp yolov10n.onnx /content/drive/MyDrive/

## Section 10 — Unit tests & reproducibility
Example: a short pytest test ensuring ONNX loads and that a basic forward pass runs (uses a random input for smoke test).

```python
# tests/test_onnx_load.py
import onnx
import onnxruntime as ort


def test_onnx_load_and_session():
    model = onnx.load('yolov10n.onnx')
    onnx.checker.check_model(model)
    sess = ort.InferenceSession('yolov10n.onnx')
    assert 'CPUExecutionProvider' in sess.get_providers()
```

Run tests with `pytest -q` after saving the file.

## Section 11 — Running & debugging in VS Code
- You can open this notebook in VS Code and run cells interactively.
- Use the Jupyter extension or the built-in Notebook UI.
- To debug cells in VS Code set breakpoints in the cell and use the 'Run by line' or the Debug Cell action.

**Open in Colab**: you can upload this notebook to Colab or use `Open in Colab` links if you publish the repo.

---
## Troubleshooting & Tips
- If export fails with errors from `torch` or `ultralytics`, run the export on an x86 machine or use Colab GPU runtime (this notebook).
- Try opset 11/12 if you see unsupported operator errors.
- If `onnx.checker` fails, try exporting without `simplify=True` to inspect raw errors.
- Use `onnxruntime` CPU provider for quick checks; GPU provider requires additional setup.

If you want, run the cells now. If you'd like, I can also add a short example to run inference with the ONNX model on a sample image inside this notebook.

## Optional: Run an ONNX inference example on a sample image
Upload a sample image and run a small inference + visualization step to verify the exported ONNX works on a real image.

Notes:
- The ONNX output format may vary between model exports. The cell below attempts a robust parsing for common YOLO-style outputs (xywh + confidence + class scores). If your model uses a different layout, adapt the parsing accordingly.

In [ ]:
# Select a sample image: prefer the repo-provided sample if available
import os
sample_b64 = 'data/sample_image.b64'
sample_png = 'data/sample_image.png'

if os.path.exists(sample_png):
    img_path = sample_png
    print('Using repo sample image:', img_path)
elif os.path.exists(sample_b64):
    print('Extracting sample image from base64...')
    # run the helper to write the PNG
    import subprocess
    subprocess.run(["python", "scripts/extract_sample_image.py"], check=False)
    if os.path.exists(sample_png):
        img_path = sample_png
        print('Extracted and using:', img_path)
    else:
        print('Extraction failed; falling back to upload prompt')
        from google.colab import files
        uploaded = files.upload()
        if len(uploaded) == 0:
            raise SystemExit('No image uploaded.')
        img_path = next(iter(uploaded.keys()))
        print('Using uploaded image:', img_path)
else:
    from google.colab import files
    uploaded = files.upload()
    if len(uploaded) == 0:
        raise SystemExit('No image uploaded.')
    img_path = next(iter(uploaded.keys()))
    print('Using uploaded image:', img_path)

# Show the image
from PIL import Image
Image.open(img_path).resize((400,400))

In [ ]:
# Run ONNX inference and visualize detections
import cv2
import numpy as np
import onnxruntime as ort
import matplotlib.pyplot as plt

# Load image
img_bgr = cv2.imread(img_path)
if img_bgr is None:
    raise SystemExit('Failed to read image')
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
orig_h, orig_w = img_rgb.shape[:2]

# Settings
input_size = 640  # match the size used for export
conf_thresh = 0.4

# Preprocess: resize (no letterbox for simplicity) and normalize
img_resized = cv2.resize(img_rgb, (input_size, input_size))
input_tensor = img_resized.astype(np.float32) / 255.0
input_tensor = np.transpose(input_tensor, (2, 0, 1))[None, :]

# Run session
sess = ort.InferenceSession('yolov10n.onnx')
input_name = sess.get_inputs()[0].name
outputs = sess.run(None, {input_name: input_tensor})
print('Number of outputs:', len(outputs))
for i, out in enumerate(outputs):
    print(f'output[{i}] shape:', out.shape)

# Try to parse the first output into detections (common YOLO layouts)
out = outputs[0]
if out.ndim == 3:  # (1, N, C)
    dets = out[0]
elif out.ndim == 2:  # (N, C)
    dets = out
else:
    print('Unexpected ONNX output shape; raw output shown above. Inspect and adapt parsing.')
    dets = None

coco_names = [
    'person','bicycle','car','motorcycle','airplane','bus','train','truck','boat','traffic light','fire hydrant','stop sign','parking meter','bench','bird','cat','dog','horse','sheep','cow','elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase','frisbee','skis','snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard','tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich','orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','couch','potted plant','bed','dining table','toilet','tv','laptop','mouse','remote','keyboard','cell phone','microwave','oven','toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear','hair drier','toothbrush'
]

vis = img_rgb.copy()
if dets is not None and dets.shape[1] >= 6:
    # Interpret as: x_center, y_center, w, h, conf, class_scores... or x,y,w,h,conf,class_id
    x = dets[:, 0]
    y = dets[:, 1]
    w = dets[:, 2]
    h = dets[:, 3]
    conf = dets[:, 4]

    if dets.shape[1] > 6:
        class_scores = dets[:, 5:]
        class_idx = np.argmax(class_scores, axis=1)
        class_score = np.max(class_scores, axis=1)
        final_score = conf * class_score
    else:
        class_idx = dets[:, 5].astype(int)
        final_score = conf

    # If coordinates are normalized (<=1), convert to pixels in resized image
    if np.max(w) <= 1.0 + 1e-6:
        x *= input_size
        y *= input_size
        w *= input_size
        h *= input_size

    x1 = x - w / 2
    y1 = y - h / 2
    x2 = x + w / 2
    y2 = y + h / 2

    # Scale back to original image size
    sx = orig_w / input_size
    sy = orig_h / input_size
    x1 *= sx; x2 *= sx; y1 *= sy; y2 *= sy

    # Filter and draw
    indices = np.where(final_score > conf_thresh)[0]
    print(f'Filtered {len(indices)} detections (score > {conf_thresh})')
    for i in indices:
        xa, ya, xb, yb = int(x1[i]), int(y1[i]), int(x2[i]), int(y2[i])
        cls = int(class_idx[i]) if class_idx is not None else -1
        label = coco_names[cls] if (0 <= cls < len(coco_names)) else str(cls)
        sc = float(final_score[i])
        cv2.rectangle(vis, (xa, ya), (xb, yb), (0, 255, 0), 2)
        cv2.putText(vis, f'{label} {sc:.2f}', (xa, max(ya - 6, 0)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,0), 2)

    plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(vis)
    plt.show()
else:
    print('Detections array missing or not in expected format. See printed output shapes and adapt parsing as needed.')